In [1]:
import pandas
pandas.set_option('display.max_colwidth', None)
import json
import logging
import os
import numpy as np
import xarray as xr

from agroservices import IPM
from weatherdata.settings import pathCache
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfea


class WeatherDataHub:    
    """
        Allows to access at IPM weather resources 
        give the list of weather adapter (resources) available on IPM and allows access to weather data source
        
        ..doctest::
        >>> wsh = WeatherDataHub()
        >>> wsh.list_resources()
        >>> wsh.get_resource(name = 'Finnish Meteorological Institute measured data')

    """

    def __init__(self):
        """
            Give an access to IPM interface from agroservice
        """
        self.ipm = IPM()
        self.sources = None
    
    @property
    def __resources__(self):
        
        if self.sources is None:
            self.sources= self.ipm.get_weatherdatasource()
        
        return {item["name"]:item for item in self.sources}
    
    @property
    def list_resources(self):
        """ display a dataframe of list of resources with their description 

        Returns
        -------
        pandas.DataFrame
            name and description of available weatherdatasource on IPM service
        """   
        df= pandas.DataFrame(self.__resources__).T.reset_index()
        df.rename({"index":"name"},inplace=True)
        
        return df[["name","description","parameters"]]
                     
    
    def __forecast__(self):
        return {key:bool(value["temporal"]["forecast"])for key,value in ws.__resources__.items()}
    
    def __endpoint__(self):
        return {key: value["endpoint"] for key, value in ws.__resources__.items()}
    
        
    def get_ressource(self, name: str):
        """ Get ressource from WeatherDataSource

        Parameters
        ----------
        name : str
             name of weatherdatasource (available in list ressource)

        Returns
        -------
        WeatherDataSource
           weatherdatasource with the name of resource

        Raises
        ------
        NotImplementedError
            the resource is unknown or the name of the resource is misspelled
        """        
        keys = [item for item in self.__resources__]

        if name in keys:
            return WeatherDataSource(name,forecast=self.__forecast__()[name],endpoint=self.__endpoint__()[name])
        else:
            raise NotImplementedError("the resource is unknown or the name of the resource is misspelled")

class WeatherDataSource(WeatherDataHub):
    def __init__(self, name, forecast,endpoint):
        self.name = name
        self.forecast=forecast
        self.endpoint=endpoint
        self.sources=None
        self.ipm = IPM()
    
    @property
    def __source__(self):
        
        if self.sources is None:
            self.sources=self.__resources__
            
        source= self.sources[self.name]
        return source
          
    
    @property               
    def parameter(self):
        parameter= self.__source__["parameters"]
        return parameter
    
    @property
    def stations(self):
        if self.__source__["spatial"]["geoJSON"] is not None:
            features=json.loads(self.__source__["spatial"]["geoJSON"])['features']
            
            #recupère les infos stations dans une properties
            stations=[feature["properties"] for feature in features]
            
            #recuperation et transformation des coordonnées
            coord_tmp=[feature['geometry']['coordinates'] for feature in features]
            station_coords= [{"latitude":float(coord[0]), "longitude":float(coord[1])} for coord in coord_tmp]
            
            # ajoute les coordonnées dans stations
            for el in range(len(stations)):
                stations[el].update(station_coords[el])
            
            # affichage des stations comme dataframe    
            df= pandas.DataFrame(stations)
            return df
        else:
            print("No stations informations for this ressources")
        
    def data(self,
             parameters =[1002,3002], 
             stationId =[101104], 
             timeStart = '2020-06-12',
             timeEnd = '2020-07-03',
             timeZone = "UTC",
             altitude = [70.0],
             longitude = [14.3711],
             latitude = [67.2828],
             credentials = None,
             interval = 3600,
             display ='ds',
             varname = 'id',
             usecache =False,
            savecache =False):
        
        responses=[] 
        
        if self.forecast== False: 
            times= pandas.date_range(timeStart,timeEnd,freq='H',tz=timeZone)

            # time transformation for query format
            timeStart = times[0].strftime('%Y-%m-%dT%H:%M:%S')
            timeEnd = times[-1].strftime('%Y-%m-%dT%H:%M:%S')
            if times.tz._tzname == 'UTC':
                timeStart +='Z'
                timeEnd += 'Z'
            else:
                decstr = times[0].strftime('%z')
                decstr = decstr[:-2] + ':' + decstr[-2:]
                timeStart += decstr
                timeEnd += decstr

            interval = pandas.Timedelta(times.freq).seconds
                
            for station in stationId:
                logging.info('start connecting to station %s' % station)
                path=os.path.join(pathCache(),str(station)+'_'+str(parameters)+"_"+timeStart.split("T")[0]+"_"+timeEnd.split("T")[0]+'.json')
                
                
                if usecache and os.path.exists(path):
                    with open(path) as f:
                        data=json.load(f)
                else:        
                    data = self.ipm.get_weatheradapter(
                                    endpoint=self.endpoint,
                                    weatherStationId=station,
                                    timeStart=timeStart,
                                    timeEnd=timeEnd,
                                    interval=interval,
                                    parameters=parameters,
                                    credentials= credentials)
                    
                
                if type(data) is dict:
                    responses.append(data)
                elif type(data) is int:
                    logging.warning('HTTPError: %s for %s' %(data,station))   
                
                if savecache and type(data) is dict:
                    with open(path,'w') as f:
                        json.dump(data, f)  
                
        elif self.endpoint=="https://ipmdecisions.nibio.no/api/wx/rest/weatheradapter/dmipoint":
            stationId=None
            interval=86400
            
            times= pandas.date_range(timeStart,timeEnd,freq='D',tz=timeZone)

            # time transformation for query format
            timeStart = times[0].strftime('%Y-%m-%dT%H:%M:%S')
            timeEnd = times[-1].strftime('%Y-%m-%dT%H:%M:%S')
            if times.tz._tzname == 'UTC':
                timeStart +='Z'
                timeEnd += 'Z'
            else:
                decstr = times[0].strftime('%z')
                decstr = decstr[:-2] + ':' + decstr[-2:]
                timeStart += decstr
                timeEnd += decstr
                
            for el in range(len(latitude)):
                path=os.path.join(pathCache(),str(timeStart)+'_'+str(timeEnd)+'_'+str(parameters)+str(altitude[el])+'_'+str(latitude[el])+"_"+str(longitude[el])+'.json')
            
                if usecache and os.path.exists(path):
                    with open(path) as f:
                        data=json.load(f)
                else:
                    data= self.ipm.get_weatheradapter_forecast(
                        endpoint=self.endpoint, 
                        altitude= altitude[el],
                        latitude=latitude[el],
                        longitude=longitude[el],
                        timeStart=timeStart,
                        timeEnd=timeEnd,
                        interval=interval,
                        parameters=parameters
                        )

                if type(data) is dict:
                    responses.append(data)
                elif type(data) is int:
                    logging.warning("HTTPError:%s for %s" %(data,[altitude[el],latitude[el],longitude[el]]))
                
                if savecache and type(data) is dict:
                    with open(path,'w') as f:
                        json.dump(data, f)
        else:
            stationId=None
            timeStart==None
            timeEnd==None
            interval=None
            for el in range(len(latitude)):
                path=os.path.join(pathCache(),str(altitude[el])+'_'+str(latitude[el])+"_"+str(longitude[el])+'.json')
            
                if usecache and os.path.exists(path):
                    with open(path) as f:
                        data=json.load(f)
                else:
                    data= self.ipm.get_weatheradapter_forecast(
                        endpoint=self.endpoint, 
                        altitude= altitude[el],
                        latitude=latitude[el],
                        longitude=longitude[el])

                if type(data) is dict:
                    responses.append(data)
                elif type(data) is int:
                    logging.warning("HTTPError:%s for %s" %(data,[altitude[el],latitude[el],longitude[el]]))
                
                if savecache and type(data) is dict:
                    with open(path,'w') as f:
                        json.dump(data, f)
        
        if display=="ds":
            return self.__convert_xarray_dataset__(responses,stationId,varname,display)
        else:
            return responses
    
    def __convert_xarray_dataset__(self, responses,stationId,varname,display):
        
        if display != "ds":
            return responses
        else:
            # times = pandas.date_range(
            #     start=responses[0]['timeStart'], 
            #     end=responses[0]['timeEnd'], 
            #     freq="H",
            #     name="time")
            
            times=pandas.date_range(start=responses[0]["timeStart"],
                                    end=responses[0]["timeEnd"],
                                    freq=str(responses[0]["interval"])+"S",
                                    name="time")
            
            #times.strftime('%Y-%m-%dT%H:%M:%S')
            
            datas= [np.array(response['locationWeatherData'][0]['data']).astype("float") for response in responses]
            
            dats = [[data[:,i].reshape(data.shape[0],1) for i in range(data.shape[1])] for data in datas]
            

            # construction of dict for dataset variable
            data_vars=[{str(response['weatherParameters'][i]):(['time','location'],dat[i]) for i in range(len(response['weatherParameters']))} for response in responses for dat in dats]

            # construction dictionnaire coordonnée
            if stationId:
                coords=[{'time':times.values,
                'location':([stationId[el]]),
                'lat':("location",[float(responses[el]['locationWeatherData'][0]['latitude'])]),
                'lon':("location",[float(responses[el]['locationWeatherData'][0]['longitude'])]),
                #'alt':[float(responses[el]['locationWeatherData'][0]['altitude'])]
                } 
                for el in range(len(responses))]
                
            else:
                coords=[{'time':times.values,
                        'location':([str([responses[el]['locationWeatherData'][0]['latitude'],responses[el]['locationWeatherData'][0]['longitude']])]),
                        'lat':('location',[responses[el]['locationWeatherData'][0]['latitude']]),
                        'lon':('location',[responses[el]['locationWeatherData'][0]['longitude']]),
            #'alt':[float(responses[el]['locationWeatherData'][0]['altitude'])]
            } for el in range(len(responses))]
                
                    
            # list de ds
            list_ds=[xr.Dataset(data_vars[el], coords=coords[el]) for el in range(len(responses))]
            
            #merge ds
            ds=xr.merge(list_ds)
            
            #add coordinates attributes
            if stationId:
                ds.coords['time'].attrs["name"]="time"
                #ds.coords['location'].attrs['name']= 'WeatherStationId'
                ds.coords['lat'].attrs['name']='latitude'
                ds.coords['lat'].attrs['unit']='degrees_north'
                ds.coords['lon'].attrs['name']='longitude'
                ds.coords['lon'].attrs['unit']='degrees_east'
            else:
                #ds.coords['location'].attrs['name']='[latitude,longitude]'
                ds.coords['lat'].attrs['name']='latitude'
                ds.coords['lat'].attrs['unit']='degrees_north'
                ds.coords['lon'].attrs['name']='longitude'
                ds.coords['lon'].attrs['unit']='degrees_east'
            
            # add data variable  attributes
            param = self.ipm.get_parameter()
            p={str(item['id']): item for item in param if item['id'] in responses[0]['weatherParameters']}
                    
                
            for el in list(ds.data_vars):
                try:
                    ds.data_vars[el].attrs=p[str(el)]
                except KeyError as e:
                    logging.exception("The weatherParameter not implemented; key error: %s".format(e))
            
            # Attribute of dataset
            if stationId:
                ds.attrs['weatherRessource']=self.name
                ds.attrs['weatherStationId']=stationId
                ds.attrs['timeStart']=str(ds.coords['time'].values[0])
                ds.attrs['timeEnd']=str(ds.coords['time'].values[-1])
                ds.attrs['parameters']=list(ds.data_vars)
            else:
                ds.attrs['weatherRessource']=self.name
                ds.attrs['timeStart']=str(ds.coords['time'].values[0])
                ds.attrs['timeEnd']=str(ds.coords['time'].values[-1])
                ds.attrs['parameters']=list(ds.data_vars)
            
            if varname=="name":
                ds= ds.rename_vars(name_dict={str(ds[el].attrs['id']):ds[el].attrs['name'] for el in list(ds.keys())}) 
                
        return ds
    
        
                
                

In [4]:
ws=WeatherDataHub()
ws.list_resources
dmi=ws.get_ressource(name="DMI Pointweather service")
rep=dmi.data(parameters=[2001,1112],timeStart="2021-09-30",timeEnd="2021-10-19",latitude=[56.488],longitude=[9.583],altitude=[0],display="ds")
rep

WARNING [agroservices:IPM:116]:  status is not ok with Internal Server Error
WARNING [agroservices:IPM:116]:  status is not ok with Internal Server Error
WARNING [agroservices:IPM:116]:  status is not ok with Internal Server Error
WARNING [agroservices:IPM:116]:  status is not ok with Internal Server Error
WARNING [agroservices:IPM:116]:  status is not ok with Internal Server Error
WARNING [agroservices:IPM:116]:  status is not ok with Internal Server Error


IndexError: list index out of range

In [ ]:

ws=WeatherDataHub()

ws.list_resources
fmi=ws.get_ressource(name='Finnish Meteorological Institute measured data')
list_station=[int(id) for id in fmi.stations.id]
list_station.remove(137188) # HTTPerror 500
list_station.remove(101649) # diff of length data
list_station.remove(855522) #HTTPerror 500
list_station.remove(137189)
list_station.remove(126737)
ds=fmi.data(parameters=[1002,3002],stationId=[101533],display="ds",varname="id",usecache=True, savecache=True)



In [ ]:
fmi.parameter


In [ ]:
import hvplot.xarray
import matplotlib.dates as mdates

def plot(ds,varname, location=None,resample=None,date=None):
    
    if resample:
        data=ds[varname].resample(time=resample).mean()
        
    else:
        data=ds[varname]
        
    if location is not None:  
        data_loc= data.sel(location=location)
        data_loc.plot.line(x="time")
        # fig = plt.figure()
        # ax = fig.add_subplot() 
        # ax.plot(x=data_loc.time,y=data_loc)
    else:
        data.plot.line(x="time")
    
    if date is not None:
        data_time = data.sel(time=date)
        plt.bar(x=data_time.location.astype("str"), height=data_time.values)
        
    return data  
        
        
    
plot(ds=ds,varname="1002",resample="D",date=None,location=[101533, 101185])


In [ ]:

def station_plot(ds,varname=None,time=None,resample=None):
    ext_lat_min=int(ds.lat.values.min()-1)
    ext_lat_max=int(ds.lat.values.max()+1)
    ext_lon_min=int(ds.lon.values.min()-1)
    ext_lon_max=int(ds.lon.values.max()+1)
    if resample:
        ds = ds.resample(time=resample).mean()
    else:
        ds = ds
        
    df=ds.to_dataframe()
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
    ax.add_feature(cfea.LAKES, zorder=3)
    ax.add_feature(cfea.OCEAN, zorder=1)
    ax.add_feature(cfea.COASTLINE, zorder=2)
    ax.add_feature(cfea.LAND, zorder=1)
    ax.add_feature(cfea.BORDERS, zorder=4)
    ax.set_extent([ext_lon_min,ext_lon_max,ext_lat_min,ext_lat_max])
    gl=ax.gridlines(draw_labels=True, zorder = 5)
    gl.right_labels = False
    gl.top_labels = False
    if (varname and time) is None:
        ax.scatter(x=df["lon"].values,y=df["lat"].values,transform=ccrs.PlateCarree(),color='k',s=10)
    else:
        ds.isel(time=time).plot.scatter(x='lon',y='lat',hue=varname,
                             ax=ax,cmap='inferno',vmin=-5,vmax=25,
                             transform=ccrs.PlateCarree(),marker='s',s=50, add_guide=True ,zorder=6)
    #ax.text(x=df["lon"].values,y=df["lat"].values,s=df.index.get_level_values("location"),color="k")
    # ax.text(x=float(ds.isel(time=0).lon.values)-0.5,y=float(ds.isel(time=0).lat.values),s=float(ds['1002'].isel(time=0).values),color="red")
    # ax.text(x=float(ds.isel(time=0).lon.values)-0.5,y=float(ds.isel(time=0).lat.values)-0.2,s=float(ds['3002'].isel(time=0).values),color="blue")

station_plot(ds, varname='1002',time=24,resample=None)


In [ ]:
# plot temperature at a given time step
ext_lat_min=int(ds.lat.values.min()-20)
ext_lat_max=int(ds.lat.values.max()+20)
ext_lon_min=int(ds.lon.values.min()-20)
ext_lon_max=int(ds.lon.values.max()+20)

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())
ax.add_feature(cfea.LAKES, zorder=3)
ax.add_feature(cfea.OCEAN, zorder=1)
ax.add_feature(cfea.COASTLINE, zorder=2)
ax.add_feature(cfea.LAND, zorder=1)
ax.add_feature(cfea.BORDERS, zorder=4)
#ax.set_extent([ext_lon_min,ext_lon_max,ext_lat_min,ext_lat_max])
gl=ax.gridlines(draw_labels=True, zorder = 5)
gl.right_labels = False
gl.top_labels = False
ds.isel(time=50).plot.scatter(x='lon',y='lat',hue='1002',
                             ax=ax,cmap='inferno',vmin=-5,vmax=25,
                             transform=ccrs.PlateCarree(),marker='s',s=50, add_guide=True ,zorder=6)

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
import matplotlib.pyplot as plt
import numpy as np

from metpy.cbook import get_test_data
from metpy.interpolate import (interpolate_to_grid, remove_nan_observations,
                               remove_repeat_coordinates)
ds.isel(time=0)

In [ ]:
def basic_map(ds, proj, title):
    """Make our basic default map for plotting"""
    ext_lat_min=ds.lat.values.min()-1
    ext_lat_max=ds.lat.values.max()+1
    ext_lon_min=ds.lon.values.min()-1
    ext_lon_max=ds.lon.values.max()+1
    fig = plt.figure(figsize=(15, 10))
    view = fig.add_axes([0, 0, 1, 1], projection=proj)
    view.set_title(title)
    view.set_extent([ext_lon_min,ext_lon_max,ext_lat_min,ext_lat_max])
    view.add_feature(cfeature.STATES.with_scale('50m'))
    view.add_feature(cfeature.OCEAN)
    view.add_feature(cfeature.COASTLINE)
    view.add_feature(cfeature.BORDERS, linestyle=':')
    return fig, view

def station_test_data(ds, variable_names, proj_from=None, proj_to=None):
    
    value = ds[variable_names].isel(time=0).values
    lon = ds.lon.values
    lat = ds.lat.values

    if proj_from is not None and proj_to is not None:
        proj_points = proj_to.transform_points(proj_from, lon, lat)
        return proj_points[:, 0], proj_points[:, 1], value

    return lon, lat, value


from_proj = ccrs.PlateCarree(central_longitude=ds.lon.values.mean())
to_proj = ccrs.PlateCarree(central_longitude=ds.lon.values.mean())
levels = list(range(-20, 20, 1))
cmap = plt.get_cmap('magma')
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)


x, y, temp = station_test_data(ds,'1002', from_proj, to_proj)
# x, y, temp = remove_nan_observations(x, y, temp)
# x, y, temp = remove_repeat_coordinates(x, y, temp)
x,y,temp

In [ ]:
x, y, temp = remove_nan_observations(x, y, temp)
#x, y, temp = remove_repeat_coordinates(x, y, temp)

temp

In [ ]:
gx, gy, img = interpolate_to_grid(x, y, temp, interp_type='cressman', minimum_neighbors=1,
                                  hres=75000, search_radius=100000)
img = np.ma.masked_where(np.isnan(img), img)
fig, view = basic_map(ds,to_proj, 'Linear')
mmb = view.pcolormesh(gx, gy, img, cmap=cmap, norm=norm)
fig.colorbar(mmb, shrink=.4, pad=0, boundaries=levels)

In [ ]:
norway=ws.get_ressource(name='Met Norway Locationforecast')
norway.data(latitude=[67.2828, 61.27], longitude=[14.3711,25.52], altitude=[70,70],display='ds',varname='id')


